***
# Robust Journey Planning

**Link to project presentation:** https://drive.google.com/file/u/2/d/1JT5YS6-EReK9E0qljRGpoRuwZnPlfxE-/view?usp=sharing&pli=1

***

## Table of Contents
* [0. Imports](#imports)
    * [0.1 HDFS/Hive](#hive)
    * [0.2 Spark](#spark)
    * [0.3 Geospatial User Defined Functions](#udf)
* [1. Data](#data)
    * [1.1 Timetable](#timetablegeostops)
    * [1.2 Actual Data](#actualdata)
    * [1.3 Geo Shapes](#geoshapes)
    * [1.4 Weather Data](#weather)
* [2. Data Preprocessing](#datapreprocessing)
    * [2.1 Preprocessing Timetable & Geostops](#preprocessingtimetablegeostops)
    * [2.2 Preprocessing Istdaten Data](#preprocessingactualdata)
* [3. Building the Transportation Graph](#transportationgraph)
* [4. Finding the Fastest Route](#findingfastestroute)
* [5. Modelling Delays](#modellingdelays)
* [6. Graphical User Interface](#gui)

## 0. Imports <a class="anchor" id="imports"></a>
In this section we import necessary packages, connect to HDFS / Hive and initialize the Spark environment we will use in the assignment. Finally, we will add support for Geospatial User Defined Funtions.

In [2]:
# INSERT A REGION OBJECTID
OBJECTID = 1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Cluster-based imports
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.sql import SparkSession, Row, HiveContext, Window,  functions as F
from pyspark.sql.types import IntegerType, StringType, ArrayType, StructField, StructType

import getpass
import os
import pandas as pd

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
%%local
# Local imports for visualization and data manipulation
from datetime import datetime
from pyarrow.fs import HadoopFileSystem, FileSelector
from pyhive import hive
from tqdm import tqdm

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import pickle
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

### 0.1 HDFS/Hive <a class="anchor" id="hive"></a>


In [4]:
%%local
# Environment variables setup
default_db = 'com490'
hive_server = os.environ.get('HIVE_SERVER', 'iccluster080.iccluster.epfl.ch:10000')
hadoop_fs = os.environ.get('HADOOP_DEFAULT_FS', 'hdfs://iccluster067.iccluster.epfl.ch:8020')
hdfs = HadoopFileSystem.from_uri(hadoop_fs)
username = os.environ.get('USER', 'anonym')
hive_host, hive_port = hive_server.split(':')

# Connect to Hive
conn = hive.connect(host=hive_host, port=int(hive_port), username=username)
cur = conn.cursor()

# Print connection details
print(f"Hadoop HDFS URL: {hadoop_fs}")
print(f"Username: {username}")
print(f"Connected to Hive at: {hive_host}:{hive_port}")

log4j:WARN No appenders could be found for logger (org.apache.hadoop.fs.FileSystem).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Hadoop HDFS URL: hdfs://iccluster067.iccluster.epfl.ch:8020
Username: hogenhau
Connected to Hive at: iccluster080.iccluster.epfl.ch:10000


In [5]:
%%local 

# Create directory for finalproject if it doesn't already exist
base_path = f"/user/{username}/"

# List all files and directories in the base path
selector = FileSelector(base_path, recursive=False)
file_info_list = hdfs.get_file_info(selector)

# Directory to check
target_directory = "finalproject"

# Check if the target directory exists among the listed files/directories
directory_exists = any(info.path.rstrip('/').split('/')[-1] == target_directory for info in file_info_list)

if not directory_exists:
    # Create the directory if it does not exist
    directory_path = f"{base_path}{target_directory}"
    hdfs.create_dir(directory_path, recursive=True)
    print(f"Directory created: {directory_path}")
else:
    print(f"Directory already exists: {base_path}{target_directory}")

Directory already exists: /user/hogenhau/finalproject


In [6]:
%%local
# Create a new database
query = f"CREATE DATABASE IF NOT EXISTS {username} LOCATION '/user/{username}/finalproject'"
cur.execute(query)
print(f"Database {username} created or already exists.")

# Switch to the new database
query = f"USE {username}"
cur.execute(query)
print(f"Switched to database: {username}")

Database hogenhau created or already exists.
Switched to database: hogenhau


In [7]:
%%local
cur.execute(f"SHOW TABLES IN {username}")
cur.fetchall()

[('geo_shapes',),
 ('sbb_csv',),
 ('sbb_csv_one_month',),
 ('sbb_istdaten_latest_parquet',),
 ('sbb_orc',),
 ('sbb_orc_istdaten',),
 ('sbb_orc_one_day',),
 ('sbb_orc_stops',),
 ('sbb_parquet',),
 ('sbb_stop_times_region',),
 ('sbb_stop_to_stop_region',),
 ('sbb_stops_lausanne',),
 ('sbb_stops_region',),
 ('twitter',),
 ('twitter_one_day',)]

In [8]:
%%local

# Make sure to give rw access to Hive and Livy
!hdfs dfs -setfacl -R -m user:hive:rwx /user/${USER}/finalproject
!hdfs dfs -setfacl -R -m default:user:hive:rwx /user/${USER}/finalproject
!hdfs dfs -setfacl -R -m user:livy:rwx /user/${USER}/finalproject
!hdfs dfs -setfacl -R -m default:user:livy:rwx /user/${USER}/finalproject

In [9]:
# Remember, when not using %%local our username is 'livy'
local_username = os.environ.get('USER', getpass.getuser())
local_username

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'livy'

In [10]:
%%local
username

'hogenhau'

### 0.2 Spark <a class="anchor" id="Spark"></a>

In [11]:
# Initializing the spark session and sending %%local {username} to Spark
sparkSession = SparkSession.builder.appName('final-project-{0}'.format(getpass.getuser())).getOrCreate()
sparkSession.getActiveSession()
print(sparkSession.getActiveSession())
print(type(sparkSession))
sc = sparkSession.sparkContext

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.session.SparkSession'>

In [12]:
%%send_to_spark -i username -t str -n username

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'username' as 'username' to Spark kernel

In [13]:
# Check that Spark has access to personal HDFS
spark.sql(f"SHOW TABLES IN {username}").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
| hogenhau|sbb_istdaten_late...|      false|
| hogenhau|             sbb_csv|      false|
| hogenhau|             sbb_orc|      false|
| hogenhau|         sbb_parquet|      false|
| hogenhau|  sbb_stops_lausanne|      false|
| hogenhau|   sbb_csv_one_month|      false|
| hogenhau|     sbb_orc_one_day|      false|
| hogenhau|             twitter|      false|
| hogenhau|     twitter_one_day|      false|
| hogenhau|    sbb_stops_region|      false|
| hogenhau|sbb_stop_to_stop_...|      false|
| hogenhau|sbb_stop_times_re...|      false|
| hogenhau|       sbb_orc_stops|      false|
| hogenhau|    sbb_orc_istdaten|      false|
| hogenhau|          geo_shapes|      false|
+---------+--------------------+-----------+

In [14]:
%%send_to_spark -i hadoop_fs -t str -n hadoop_fs

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'hadoop_fs' as 'hadoop_fs' to Spark kernel

### 0.3 Geospatial User Defined Functions <a class="anchor" id="udf"></a>


In [15]:
spark.sql(f"""
ADD JARS
    {hadoop_fs}/data/jars/esri-geometry-api-2.2.4.jar
    {hadoop_fs}/data/jars/spatial-sdk-hive-2.2.0.jar
    {hadoop_fs}/data/jars/spatial-sdk-json-2.2.0.jar
""")

# Create or replace temporary functions
gis_functions = [
    "ST_Point", "ST_Distance", "ST_SetSRID", "ST_GeodesicLengthWGS84",
    "ST_LineString", "ST_AsBinary", "ST_PointFromWKB", "ST_GeomFromWKB", "ST_Contains"
]

for func in gis_functions:
    spark.sql(f"CREATE OR REPLACE TEMPORARY FUNCTION {func} AS 'com.esri.hadoop.hive.{func}'")

# Get the list of functions
functions_df = spark.sql("SHOW FUNCTIONS")

# Filter and show only functions starting with 'st_'
functions_df.filter(F.col("function").startswith("st_")).show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+
|function              |
+----------------------+
|st_asbinary           |
|st_contains           |
|st_distance           |
|st_geodesiclengthwgs84|
|st_geomfromwkb        |
|st_linestring         |
|st_point              |
|st_pointfromwkb       |
|st_setsrid            |
+----------------------+

## 1. Data <a class="anchor" id="data"></a>

In this section, we will load and preprocess first the timetable and geostops data and finally the Istdaten data.

In [16]:
# Just to check that everything looks normal
print(f"remote USER={os.getenv('USER',None)}")
print(f"local USER={username}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

remote USER=livy
local USER=hogenhau

In [17]:
%%local
print(f"local USER={os.getenv('USER',None)}")

local USER=hogenhau


### 1.1 Timetables<a class="anchor" id="timetables"></a>


In [18]:
# Load timetables data from HDFS
stops = spark.read.orc('/data/sbb/orc/timetables/stops/year=2024/month=5/day=16')
stop_times = spark.read.orc('/data/sbb/orc/timetables/stop_times/year=2024/month=5/day=16')
trips = spark.read.orc('/data/sbb/orc/timetables/trips/year=2024/month=5/day=16')
calendar = spark.read.orc('/data/sbb/orc/timetables/calendar/year=2024/month=5/day=16')
routes = spark.read.orc('/data/sbb/orc/timetables/routes/year=2024/month=5/day=16')
transfers = spark.read.orc('/data/sbb/orc/timetables/transfers/year=2024/month=5/day=16')

stops.show(2)
stop_times.show(2)
trips.show(2)
calendar.show(2)
routes.show(2)
transfers.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+---------------+--------------+-------------+--------------+
|stop_id|           stop_name|       stop_lat|      stop_lon|location_type|parent_station|
+-------+--------------------+---------------+--------------+-------------+--------------+
|1100008|Zell (Wiesental),...|47.710084270235|7.859647882747|             | Parent1100008|
|1100009|Zell (Wiesental),...|47.713191104479|7.862908767228|             | Parent1100009|
+-------+--------------------+---------------+--------------+-------------+--------------+
only showing top 2 rows

+--------------------+------------+--------------+-----------+-------------+-----------+-------------+
|             trip_id|arrival_time|departure_time|    stop_id|stop_sequence|pickup_type|drop_off_type|
+--------------------+------------+--------------+-----------+-------------+-----------+-------------+
|5874.TA.91-14-D-j...|    19:06:00|      19:06:00|8587057:0:I|           13|          0|            0|
|5874.TA.91-14-D-

In [19]:
transfers.show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------+-------------+-----------------+
|from_stop_id| to_stop_id|transfer_type|min_transfer_time|
+------------+-----------+-------------+-----------------+
|     1100079|    8014441|            2|              240|
|     1100079|8014441:0:2|            2|              240|
|     1100079|8014441:0:3|            2|              240|
|     1100079|8014441:0:1|            2|              240|
|     1100084|    8014440|            2|              180|
|     1100084|8014440:0:1|            2|              180|
|     1100097|    8014447|            2|              240|
|     1100097|8014447:0:2|            2|              240|
|     1100097|8014447:0:1|            2|              240|
|     1100097|8014447:0:3|            2|              240|
|     1100102|    8014446|            2|              240|
|     1100102|8014446:0:1|            2|              240|
|     1100158|    8014445|            2|              180|
|     1100158|8014445:0:1|            2|              18

### 1.2 Istdaten <a class="anchor" id="istdaten"></a>

In [20]:
istdaten = spark.read.orc('/data/sbb/orc/istdaten')

#istdaten.printSchema()
istdaten.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------------+------------+-------------+--------------+----------+---------+-----------+---------+-------------------+--------------+-------------+-------+--------------------+----------------+-------------------+------------------+----------------+-------------------+------------------+-------------+----+-----+
|betriebstag|fahrt_bezeichner|betreiber_id|betreiber_abk|betreiber_name|produkt_id|linien_id|linien_text|umlauf_id|verkehrsmittel_text|zusatzfahrt_tf|faellt_aus_tf|  bpuic|   haltestellen_name|    ankunftszeit|        an_prognose|an_prognose_status|    abfahrtszeit|        ab_prognose|ab_prognose_status|durchfahrt_tf|year|month|
+-----------+----------------+------------+-------------+--------------+----------+---------+-----------+---------+-------------------+--------------+-------------+-------+--------------------+----------------+-------------------+------------------+----------------+-------------------+------------------+-------------+----+-----+
| 22.12

### 1.3 Geo Shapes <a class="anchor" id="geoshapes"></a>


In [21]:
spark.sql(f"DROP TABLE IF EXISTS {username}.geo_shapes")
spark.sql(f"""
CREATE EXTERNAL TABLE {username}.geo_shapes(
    objectid INT,
    name     STRING,
    geometry BINARY
)
PARTITIONED BY(country STRING, region STRING)
ROW FORMAT SERDE 'com.esri.hadoop.hive.serde.GeoJsonSerDe' 
STORED AS INPUTFORMAT 'com.esri.json.hadoop.UnenclosedEsriJsonInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION '/data/geo/json/'
""")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [22]:
spark.sql(f"MSCK REPAIR TABLE {username}.geo_shapes")
geo_shapes = spark.sql(f"SELECT * FROM {username}.geo_shapes")
geo_shapes.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------+--------------------+-------+------+
|objectid|          name|            geometry|country|region|
+--------+--------------+--------------------+-------+------+
|     109|           Uri|[00 00 10 E6 03 1...|     CH|    UR|
|    1016|    Schattdorf|[00 00 10 E6 03 1...|     CH|    UR|
|    1124|  Altdorf (UR)|[00 00 10 E6 03 1...|     CH|    UR|
|    1177|      Isenthal|[00 00 10 E6 03 1...|     CH|    UR|
|    1295|  Seedorf (UR)|[00 00 10 E6 06 1...|     CH|    UR|
|    1344|     Andermatt|[00 00 10 E6 03 1...|     CH|    UR|
|    1460|UnterschÃ¤chen|[00 00 10 E6 03 1...|     CH|    UR|
|    1592|  Attinghausen|[00 00 10 E6 03 1...|     CH|    UR|
|    1646|    GÃ¶schenen|[00 00 10 E6 03 1...|     CH|    UR|
|    1686|     Hospental|[00 00 10 E6 03 1...|     CH|    UR|
+--------+--------------+--------------------+-------+------+
only showing top 10 rows

### 1.4 Optional: Weather Data <a class="anchor" id="weatherdata"></a>

In [23]:
# Add code from hw3
#stations = spark.read.csv('/data/wunderground/csv/stations', header=True)
#weather_data = spark.read.json('/data/wunderground/json/history')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 2. Data Preprocessing <a class="anchor" id="datapreprocessing"></a>

### 2.1 Preprocessing Timetable & Geostops <a class="anchor" id="preprocessingtimetablegeostops"></a>


In [24]:
stops.createOrReplaceTempView("stops")

stops_region = spark.sql(f"""
SELECT
    a.stop_id,
    a.stop_name,
    a.stop_lat,
    a.stop_lon,
    a.parent_station
FROM stops a JOIN {username}.geo_shapes b
ON ST_Contains(b.geometry, ST_Point(a.stop_lon, a.stop_lat))
WHERE b.objectid = {OBJECTID}
""")


stops_region.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+---------------+--------------+--------------+
|    stop_id|           stop_name|       stop_lat|      stop_lon|parent_station|
+-----------+--------------------+---------------+--------------+--------------+
|    8501075|Lausanne-Ouchy (lac)|46.505077550443|6.627608470405| Parent8501075|
|8501117:0:1|            Bussigny|46.547260262317|6.552428464267| Parent8501117|
|8501117:0:3|            Bussigny|46.546957528818|6.552850672450| Parent8501117|
|8501117:0:4|            Bussigny|46.546914281038|6.552805756686| Parent8501117|
|8501118:0:1|           Renens VD|46.536206306037|6.580923025083| Parent8501118|
+-----------+--------------------+---------------+--------------+--------------+
only showing top 5 rows

In [25]:
# We choose to focus only on the weekday schedule
print(f"Trips full week: {calendar.count()}")

# Filter for services that are active from Monday to Friday
# We want to filter out services that are only active in the weekends

weekend_calendar = calendar.filter(
    (F.col("monday") == False) &
    (F.col("tuesday") == False) &
    (F.col("wednesday") == False) &
    (F.col("thursday") == False) &
    (F.col("friday") == False) &
    (F.col("saturday") == True) &
    (F.col("sunday") == True)
).select("service_id").distinct()

weekend_calendar.show(2)

print(f"Trips Only active on Weekends: {weekend_calendar.count()}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Trips full week: 38694
+----------+
|service_id|
+----------+
|  TA+24k40|
|  TA+2s440|
+----------+
only showing top 2 rows

Trips Only active on Weekends: 3670

In [26]:
# Connections table
# Register the DataFrames as temporary views
stops_region.createOrReplaceTempView("stops_region")
stops.createOrReplaceTempView("stops")
stop_times.createOrReplaceTempView("stop_times")
trips.createOrReplaceTempView("trips")
routes.createOrReplaceTempView("routes")
calendar.createOrReplaceTempView("calendar")


# Perform an anti join to filter out weekend trips
weekday_trips = trips.join(
    weekend_calendar, 
    trips.service_id == weekend_calendar.service_id, 
    "left_anti"
)

# Give alias because of ambiguous "trip_id"
stop_times_joined = stop_times.alias("st").join(
    weekday_trips.alias("wt"),
    F.col("st.trip_id") == F.col("wt.trip_id"),
    "inner"
)


final_df = stop_times_joined.join(
    stops_region.alias("sr"),
    F.col("st.stop_id") == F.col("sr.stop_id"),
    "inner"
).select(
    F.col("st.trip_id").alias("trip_id"),
    F.col("st.arrival_time").alias("arrival_time"),
    F.col("st.departure_time").alias("departure_time"),
    F.col("st.stop_id").alias("stop_id"),
    F.col("st.stop_sequence").alias("stop_sequence"),
    F.col("wt.route_id").alias("route_id"),
    F.col("sr.stop_name").alias("stop_name"),
    F.col("sr.stop_lat").alias("stop_lat"),
    F.col("sr.stop_lon").alias("stop_lon"),
    F.col("sr.parent_station").alias("parent_station")
).orderBy("trip_id", "stop_sequence")

# Add ROUTES_DESC from the routes table to include transport type
final_df = final_df.join(
    routes,
    final_df.route_id == routes.route_id,
    "left"  # Left join to keep all records from 'final_df' even if no match in 'routes'
).select(
    final_df["*"],  # Keep all existing columns from 'final_df'
    routes.route_desc  # Add the 'ROUTE_DESC' column from 'routes'
)


# Create a filter to select only the times between 06:00:00 and 20:00:00
time_filter = (F.col("departure_time") >= "06:00:00") & (F.col("departure_time") <= "20:00:00") & (F.col("arrival_time") >= "06:00:00") & (F.col("arrival_time") >= "06:00:00")

print(f"Distinct trip_ids before filtering: {final_df.select('trip_id').distinct().count()}")
final_df = final_df.filter(time_filter)
print(f"Distinct trip_ids after filtering: {final_df.select('trip_id').distinct().count()}")
final_df.show(5)

# Print headers to copy and load them manually in local pd.DataFrame
#final_df_headers = final_df.columns
#print(final_df_headers)

# Write PySpark dataframe to csv
#final_df.write.csv(f"/user/{username}/folder/final_df", header=False, mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Distinct trip_ids before filtering: 60415
Distinct trip_ids after filtering: 44013
+--------------------+------------+--------------+-----------+-------------+-------------+---------+---------------+--------------+--------------+----------+
|             trip_id|arrival_time|departure_time|    stop_id|stop_sequence|     route_id|stop_name|       stop_lat|      stop_lon|parent_station|route_desc|
+--------------------+------------+--------------+-----------+-------------+-------------+---------+---------------+--------------+--------------+----------+
|1.TA.91-1R-Y-j24-...|    18:57:00|      19:01:00|8501120:0:1|            2|91-1R-Y-j24-1| Lausanne|46.516774559699|6.629512898808| Parent8501120|       EXT|
|1.TA.91-2E-Y-j24-...|    19:45:00|      19:45:00|8501120:0:6|            1|91-2E-Y-j24-1| Lausanne|46.516521109968|6.629018825402| Parent8501120|       TGV|
|1.TA.91-2H-Y-j24-...|    13:43:00|      13:43:00|8501120:0:4|            1|91-2H-Y-j24-1| Lausanne|46.516669470930|6.629054758

In [27]:
#%%local
#!hdfs dfs -getmerge /user/{username}/folder/final_df final_df_folder/final_df_combined.csv

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
%%local
final_df = pd.read_csv("final_df_folder/final_df_combined.csv", header=None)
final_df.columns = ['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence', 'route_id', 'stop_name', 'stop_lat', 'stop_lon', 'parent_station', 'route_desc']
final_df = final_df.sort_values(by=['trip_id', 'stop_sequence'])

# Copy 'stop_id' to 'arrival_stop_id'
final_df['arrival_stop_id'] = final_df['stop_id']

# Create 'departure_stop_id' by shifting 'stop_id' within each group
final_df['departure_stop_id'] = final_df.groupby('trip_id')['stop_id'].shift(-1)

# Helper function to convert HH:MM:SS time string to seconds past midnight
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h * 3600 + m * 60 + s

# Apply the time conversion to both arrival and departure times
final_df['arrival_time_seconds'] = final_df['arrival_time'].apply(time_to_seconds)
final_df['departure_time_seconds'] = final_df['departure_time'].apply(time_to_seconds)

# Calculate 'travel_time_seconds' as the difference between the 'arrival_time_seconds' of the next stop
# and 'departure_time_seconds' of the current stop, within each trip
final_df['travel_time_seconds'] = final_df['arrival_time_seconds'] - final_df.groupby('trip_id')['departure_time_seconds'].shift(1)

# Now you can check your dataframe
final_df.head(6)

Output()

In [41]:
%%local 
df = pd.read_pickle("./final_df.pkl")
df.head(6)

Output()

In [29]:
%%local
print(len(final_df.trip_id.unique()))
print(len(final_df.route_id.unique()))
print(len(final_df.stop_id.unique()))
final_df.to_pickle("./final_df.pkl")

44013
89
544


In [30]:
stop_pairs_within_500m = spark.sql("""
SELECT 
    a.stop_id AS stop_id1,
    b.stop_id AS stop_id2,
    a.stop_name AS stop_name1,
    b.stop_name AS stop_name2,
    a.stop_lat AS stop_lat1,
    a.stop_lon AS stop_lon1,
    b.stop_lat AS stop_lat2,
    b.stop_lon AS stop_lon2,
    a.parent_station AS parent_station1,
    b.parent_station AS parent_station2,
    ST_GeodesicLengthWGS84(ST_SetSRID(ST_LineString(a.stop_lon, a.stop_lat, b.stop_lon, b.stop_lat), 4326)) AS distance_meters
FROM 
    stops_region a 
CROSS JOIN 
    stops_region b
WHERE 
    a.stop_id != b.stop_id
    AND ST_GeodesicLengthWGS84(ST_SetSRID(ST_LineString(array(ST_Point(a.stop_lon, a.stop_lat), ST_Point(b.stop_lon, b.stop_lat))), 4326)) < 500
""")

# Calculate walking transfer times in minutes
# We assume that 2min mininum are required for transfers within a same location, 
# to which we add 1min per 50m walking time to connect two stops at most 500m appart
stop_pairs_within_500m = stop_pairs_within_500m.withColumn(
    "transfer_time_minutes",
    F.expr("2 + (distance_meters / 50)")
)
# Show results
stop_pairs_within_500m.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+--------------------+--------------------+---------------+--------------+---------------+--------------+---------------+---------------+-----------------+---------------------+
|stop_id1|stop_id2|          stop_name1|          stop_name2|      stop_lat1|     stop_lon1|      stop_lat2|     stop_lon2|parent_station1|parent_station2|  distance_meters|transfer_time_minutes|
+--------+--------+--------------------+--------------------+---------------+--------------+---------------+--------------+---------------+---------------+-----------------+---------------------+
| 8501075| 8591986|Lausanne-Ouchy (lac)|Lausanne, Beau-Ri...|46.505077550443|6.627608470405|46.508304989443|6.627500672571|  Parent8501075|  Parent8591986|358.8614310087467|    9.177228620174933|
| 8501075| 8592086|Lausanne-Ouchy (lac)|Lausanne, Ouchy-O...|46.505077550443|6.627608470405|46.507470325313|6.626638289898|  Parent8501075|  Parent8592086|276.2107661651952|   7.5242153233039035|
+--------+--------+-

In [ ]:
# Print headers to copy and load them manually in local pd.DataFrame
#stop_pairs_within_500m_headers = stop_pairs_within_500m.columns
#print(stop_pairs_within_500m_headers)

# Write PySpark dataframe to csv
#stop_pairs_within_500m.write.csv(f"/user/{username}/folder/stop_pairs_within_500m_folder", header=False, mode="overwrite")

In [33]:
#%%local
#!hdfs dfs -getmerge /user/{username}/folder/stop_pairs_within_500m_folder stop_pairs_within_500m_folder/stop_pairs_within_500m_df_combined.csv

In [ ]:
%%local
stop_pairs_within_500m = pd.read_csv("stop_pairs_within_500m_folder/stop_pairs_within_500m_df_combined.csv", header=None)
stop_pairs_within_500m.columns = ['stop_id1', 'stop_id2', 'stop_name1', 'stop_name2', 'stop_lat1', 'stop_lon1', 'stop_lat2', 'stop_lon2', 'parent_station1', 'parent_station2', 'distance_meters', 'transfer_time_minutes']

def minutes_to_seconds(minutes):
    return minutes * 60

stop_pairs_within_500m['transfer_time_seconds'] = stop_pairs_within_500m['transfer_time_minutes'].apply(minutes_to_seconds)
stop_pairs_within_500m.head()
#stop_pairs_within_500m.to_pickle("./stop_pairs_within_500m.pkl")

## 3. Building the Transportation Graph <a class="anchor" id="transportationgraph"></a>
- NetworkX doesn't work in the PySpark kernel. So we need to run it in `%%local`.
    - For that we need all our PySpark dataframes loaded locally.
    - **Instead, refer to our python kernel jupyter notebook**: "graph_build_and_search.ipynb"

## 4. Finding the Fastest Route <a class="anchor" id="findingfastestroute"></a>
**Refer to our python kernel jupyter notebook**: "graph_build_and_search.ipynb"

## 5. Modelling Delays <a class="anchor" id="modellingdelays"></a>
**Refer to our python kernel jupyter notebooks and .py scripts**: 
- "data_pyspark_model.ipynb"
- "model_training_and_inference_sklearn.ipynb"
- "ml.py"

## 6. Graphical User Interface <a class="anchor" id="gui"></a>
**Refer to:** interface.py